Notebook for Pre-selection of models. For this purpose a shallow train/test evaluation protocol is applied:

In [ ]:
import pandas as pd
import numpy as np
from dataloading import DataLoader
from eda_py import EDA
from eALS_adaptor import eALSAdaptor
from implicit.evaluation import train_test_split, ranking_metrics_at_k
from cv_py import CrossValidation

In [ ]:
dl = DataLoader()

In [ ]:
%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

In [ ]:
user_item_co = dl.import_data('AGCO', 'CO', 'df')
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [ ]:
user_item_filtered = dl.remove_low_interact_items(user_item_co, 1)
user_item_filtered_t = dl.remove_low_interact_items(user_item_co_t, 1)

In [ ]:
user_item_filtered_log = dl.log_scale_df(user_item_filtered, 0.01)
user_item_filtered_log_t = dl.log_scale_df(user_item_filtered_t, 0.01)

In [ ]:
user_item_csr = dl.to_csr(user_item_filtered_log)
user_item_csr_t = dl.to_csr(user_item_filtered_log_t)

In [ ]:
train, test = train_test_split(user_item_csr, 0.8, 22)
train_t, test_t = train_test_split(user_item_csr_t, 0.8, 22)

In [ ]:
cv = CrossValidation(user_item_csr, 5)
cv_t = CrossValidation(user_item_csr_t, 5)

In [ ]:
space_iALS = {'factors' : [64], 'regularization' : [60, 80, 100, 120, 140, 160], 'alpha' : [0.1, 0.2, 0.3, 0.4, 0.5], 'iterations' : [15]}
space_iALS_t = {'factors' : [64], 'regularization' : [5, 10, 20, 30, 40, 50, 60, 80, 100], 'alpha' : [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7], 'iterations' : [15]}
hyper_ials = cv.hyperp_tuning_simple(test, train, space_iALS, 'iALS')
hyper_ials_t = cv.hyperp_tuning_simple(test_t, train_t, space_iALS_t, 'iALS')

In [ ]:
space_eALS = {'factors' : [64], 'regularization' : [5, 10, 20], 'alpha' : [0.5, 1, 2], 'w0' : [20, 30, 40, 50]}
space_eALS_t = {'factors' : [64], 'regularization' : [0.5, 1, 2], 'alpha' : [0.1, 0.2, 0.3], 'w0' : [30, 40, 50, 60]}
hyper_eals = cv.hyperp_tuning_simple(test, train, space_eALS, 'eALS')
hyper_eals_t = cv.hyperp_tuning_simple(test_t, train_t, space_eALS_t, 'eALS')

In [ ]:
space_BPR = {'factors' : [64], 'regularization' : [0.001, 0.005, 0.01, 0.05, 0.1], 'learning_rate' : [0.005, 0.01, 0.03, 0.05, 0.07], 'iterations' : [15]}
space_BPR_t = {'factors' : [64], 'regularization' : [0.001, 0.005, 0.01, 0.05, 0.07, 0.1], 'learning_rate' : [0.005, 0.01, 0.03, 0.04, 0.05, 0.07], 'iterations' : [15]}
hyper_bpr = cv.hyperp_tuning_simple(test, train, space_BPR, 'BPR')
hyper_bpr_t = cv_t.hyperp_tuning_simple(test_t, train_t, space_BPR_t, 'BPR')

In [ ]:
space_LMF = {'factors' : [64], 'regularization' : [5, 10, 20, 30, 40, 50], 'learning_rate' : [0.3, 0.5, 0.7, 1, 2], 'iterations' : [15], 'neg_prop': [10, 20, 30]}
space_LMF_t = {'factors' : [64], 'regularization' : [1, 5, 10, 20, 30, 40, 50], 'learning_rate' : [0.3, 0.5, 0.7, 1, 2], 'iterations' : [15], 'neg_prop': [0.3, 0.5, 1, 2]}
hyper_lmf = cv.hyperp_tuning_simple(test, train, space_LMF, 'LMF')
hyper_lmf_t = cv.hyperp_tuning_simple(test_t, train_t, space_LMF_t, 'LMF')